# Seminar Project: Speech and Article Prompting

---------------
``` 

Jakob Amann
Elias Gabriel Heppner, 1244414
Andri Rutschmann

### Loading Packages


In [1]:
import numpy as np 
import pandas as pd 
import time
import pickle
import time
from tqdm.auto import tqdm
import nltk
# nltk.download('punkt')
from nltk.tokenize import word_tokenize # import tokenizer to estimate token count of a speech
import openai

api_keys = ["wAlKn00lAQIdM4R9nskS", "4hohJnlUY52Tqk24HXfh", '6uS6Kv0efLZLGsy0In3m'] # elias, jakob, andri
openai.api_base = "http://91.107.239.71:80"
openai.api_key = api_keys[0] # choose api key

C:\Users\Andri\envs\css\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Parameter Definition

In [13]:
# Define 
pol_party = [["Alice Weidel", "AfD"], 
    ["Friedrich Merz", "CDU"], 
    ["Olaf Scholz", "SPD"], 
    ["Agnes Strack-Zimmermann", "FDP"], 
    ["Annalena Baerbock", "Grüne"],
    ["Robert Habeck", "Grüne"],
    ["Christian Lindner", "FDP"],
    ["Saskia Esken", "SPD"],
    ["Norbert Walter-Borjans", "SPD"],
    ["Alexander Gauland", "AfD"],
    ["Tino Chrupalla", "AfD"],
    ["Janine Wissler", "Die Linke"],
    ["Dietmar Bartsch", "Die Linke"],
    ["Markus Söder", "CSU"],
    ["Gregor Gysi", "Die Linke"]
]

newspapers = ['taz', 'Zeit', 'Süddeutsche', 'Spiegel', 'Welt'] # BILD?!
topics_de = ["Waffenlieferungen an die Ukraine", "Covid-19 Restriktionen"]
topics_en = ['Weapon exports to Ukraine', 'Covid-19 Restrictions']

### Prompts


In [61]:
##Speech Prompts

#v1
def prompt_speech_v1(pol, par, top, lng):
    
    prompt_de = "Verfasse eine Rede für den deutschen Bundestag im Stil der Politikerin " + pol + " der " + par + " zum Thema " + top + "." # do we want to gender? - could potentially change the outcome. here, we have the feminine form (on purpose?)
    prompt_eng = "Write a speech for the German Parliament in the style of the " + par + " politician " + pol + " on the topic of " + top + "]. The speech should be in German."

    if lng == "de": 
        return prompt_de    
    else:
        return prompt_eng
    
    
#v2
def prompt_speech_v2(pol, par, top, lng):
    # here, I integrated a gender neutral form and specified a word count to get longer responses (also tried 5-minütige Rede - didn't work)
    # the line at the end is important to avoid explanations why that is potentially problematic etc. (works acutally quite well)
    prompt_de = "Verfasse eine ausführliche Rede mit mindestens 1000 Wörtern für den deutschen Bundestag im Stile des Mitglied des Bundestages " + pol + " der " + par + " zum Thema " + top + ". Gib ausschließlich die vollständige, erstellte Rede zurück."
    
    
    # here, we'll get back a german response - not sure, if we should translate the topic into english as well or leave it in german
    # I feel like, the responses generated with the english prompt are overall longer
    prompt_eng = "Write a detailed speech of at least 1000 words for the German Bundestag in the style of the member of the Bundestag " + pol + " from the " + par + " party on the topic of " + top + ". Return only the complete speech created."

    if lng == "de": 
        return prompt_de    
    else:
        return prompt_eng
    
    
##Article: Newspaper Prompts

#v1
    
##Article: Politicians Prompt



### Trial Runs
Using Chat Interface (not API). 

Changed the prompt (added line to get "ONLY" the speech, no explanation why thats potentially problematic and added "ausführliche" Rede to get a longer response (w/o real success)). 

Problem: Original Speech Token Count often way higher: e.g., fin_df.loc[0]['speech_content'] -> 1442 tokens
BUT, there seems to be a lot of variance as there are very short "speeches" in the data set (more like short remarks of some politicians) -> might need to define a mininum token threshold to avoid mapping irrelevant "speeches"



In [58]:
# example_GPT4_output = "Sehr geehrte Damen und Herren, heute stehe ich vor Ihnen, um ein Thema von großer Tragweite zu diskutieren – die Waffenlieferungen an die Ukraine. In einer Zeit, in der die Welt mehr denn je nach Frieden und Stabilität strebt, müssen wir die Konsequenzen unserer Handlungen sorgfältig abwägen. Unsere Entscheidungen auf der internationalen Bühne zeigen, wofür Deutschland steht. Es geht nicht nur um die Bereitstellung militärischer Unterstützung, es geht um die Prinzipien, die wir als Nation vertreten. Die Frage, die wir uns stellen müssen, lautet: Fördern wir wirklich den Frieden durch diese Waffenlieferungen, oder tragen wir zur Verlängerung eines Konflikts bei, dessen Ende nicht abzusehen ist? Wir müssen uns bewusst sein, dass jede Waffe, die wir senden, nicht nur eine militärische, sondern auch eine politische Botschaft trägt. Eine Botschaft, die weitreichende Folgen für unser Land und Europa haben kann. Die Sicherheit Europas und Deutschlands muss oberste Priorität haben. Doch wie können wir Sicherheit gewährleisten, wenn wir selbst zum Akteur in einem Konflikt werden, der sich weit über unsere Grenzen hinaus erstreckt? Es ist unerlässlich, dass wir unsere Strategie überdenken und uns auf Lösungen konzentrieren, die auf Diplomatie und Dialog basieren, statt auf militärische Eskalation. Wir sollten Brückenbauer sein, keine Lieferanten von Waffen, die den Frieden weiter hinauszögern. Lassen Sie uns nicht vergessen, dass wahre Stärke nicht in der Fähigkeit liegt, Krieg zu führen, sondern in der Fähigkeit, Frieden zu schaffen und zu erhalten. Es ist an der Zeit, dass Deutschland eine Führungsrolle übernimmt, indem es Wege zum Frieden fördert, statt die Flammen eines andauernden Konflikts zu schüren. Ich appelliere an Sie, meine Damen und Herren, die langfristigen Auswirkungen unserer Entscheidungen zu bedenken. Lassen Sie uns gemeinsam für eine Zukunft arbeiten, in der Frieden und Stabilität die Grundpfeiler unseres Handelns sind. Ich danke Ihnen."
example_GPT4_output = "Sehr geehrte Damen und Herren, heute stehe ich vor Ihnen, um ein Thema zu diskutieren, das sowohl in der deutschen Öffentlichkeit als auch in den politischen Gremien unseres Landes zu Recht kontrovers diskutiert wird: die Frage der Waffenlieferungen an die Ukraine. In diesen schwierigen Zeiten ist es von größter Bedeutung, dass wir die Konsequenzen unserer Entscheidungen sorgfältig abwägen. Die Lieferung von Waffen in ein Konfliktgebiet ist eine Entscheidung mit weitreichenden Folgen, nicht nur für die unmittelbar Beteiligten, sondern auch für die Sicherheit und Stabilität Europas und der Welt. Die Alternative für Deutschland steht für eine Politik der Vernunft und Besonnenheit. Wir sind der Überzeugung, dass Konflikte primär durch diplomatische und politische Mittel gelöst werden sollten, nicht durch die Eskalation militärischer Auseinandersetzungen. Die Geschichte hat uns gelehrt, dass Waffenlieferungen in Krisengebiete oft nicht zur Lösung der Konflikte beitragen, sondern zur Verlängerung und Intensivierung der Kämpfe führen können. Darüber hinaus müssen wir die sicherheitspolitischen Interessen Deutschlands und Europas im Auge behalten. Die Lieferung von Waffen an die Ukraine birgt das Risiko einer direkten Konfrontation mit Russland, einem Nuklearstaat. Dies würde nicht nur die Sicherheit der Ukraine, sondern auch die Sicherheit Deutschlands und Europas gefährden. Wir dürfen auch die wirtschaftlichen Konsequenzen nicht außer Acht lassen. Die aktuelle Krise hat bereits jetzt erhebliche Auswirkungen auf die Energieversorgung und die Lebenshaltungskosten in Deutschland. Eine weitere Eskalation des Konflikts würde diese Probleme nur verschärfen. Abschließend möchte ich betonen, dass die Alternative für Deutschland fest an die Notwendigkeit von Frieden und Stabilität in Europa und der Welt glaubt. Wir setzen uns für eine Politik ein, die auf Dialog und Diplomatie basiert, und lehnen Maßnahmen ab, die zu einer weiteren Eskalation der aktuellen Krise führen könnten. Ich fordere die Bundesregierung auf, sich für diplomatische Lösungen einzusetzen und von weiteren Waffenlieferungen an die Ukraine abzusehen. Lasst uns gemeinsam nach friedlichen Lösungen suchen, die zur dauerhaften Stabilität in der Region und zum Wohl aller beteiligten Völker beitragen. Vielen Dank."
tokens = word_tokenize(example_GPT4_output, language='german')
print(len(tokens))
used_prompt = 'Verfasse eine ausführliche Rede für den deutschen Bundestag im Stil der Politikerin Alice Weidel der AfD zum Thema "Waffenlieferungen an die Ukraine". Gib ausschließlich die vollständige, erstellte Rede zurück.'
example_GPT3_5_output = "Sehr geehrte Damen und Herren, als Vertreterin der Alternative für Deutschland möchte ich heute das Thema der Waffenlieferungen an die Ukraine ansprechen. Es ist von entscheidender Bedeutung, dass wir eine klare und rationale Herangehensweise an dieses Thema finden. Zunächst einmal müssen wir die Ursachen und den Kontext dieser Situation verstehen. Die Ukraine befindet sich seit Jahren in einem Konflikt mit Separatisten im Osten des Landes. Diese Separatisten werden von Russland unterstützt und haben bereits unzählige Menschenleben gefordert. Es ist daher verständlich, dass die ukrainische Regierung nach Mitteln sucht, um sich zu verteidigen und die territoriale Integrität ihres Landes zu wahren. Dennoch dürfen wir nicht die Risiken und möglichen Folgen von Waffenlieferungen außer Acht lassen. Eine Eskalation des Konflikts könnte nicht nur zu weiterem Blutvergießen führen, sondern auch die Stabilität der gesamten Region gefährden. Wir müssen uns bewusst sein, dass Waffenlieferungen eine Spirale der Gewalt in Gang setzen könnten, die schwer zu kontrollieren ist. Stattdessen sollten wir verstärkt auf diplomatische Lösungen setzen. Der Dialog und die Verhandlungen zwischen den Konfliktparteien sind der einzige Weg, um langfristigen Frieden und Stabilität in der Region zu gewährleisten. Deutschland und die internationale Gemeinschaft sollten sich aktiv dafür einsetzen, diesen Prozess zu unterstützen und zu fördern. Wir müssen auch bedenken, dass Waffenlieferungen an die Ukraine das Verhältnis zu Russland weiter belasten könnten. Eine Eskalation der Spannungen zwischen Russland und dem Westen wäre im Interesse niemandes. Wir sollten stattdessen bestrebt sein, konstruktive Beziehungen zu Russland aufzubauen und gemeinsame Lösungen für die Probleme der Region zu finden. Insgesamt müssen wir eine ausgewogene und verantwortungsvolle Herangehensweise an das Thema der Waffenlieferungen an die Ukraine verfolgen. Wir dürfen weder die Sicherheit der Ukraine vernachlässigen noch die Risiken einer Eskalation des Konflikts ignorieren. Diplomatie und Dialog sollten immer unsere bevorzugten Instrumente sein, um Konflikte zu lösen und Frieden zu sichern. Vielen Dank"
tokens = word_tokenize(example_GPT3_5_output, language='german')
print(len(tokens))

355
331
Write a speech for the German Parliament in the style of the AfD politician Alice Weidel on the topic of Waffenlierferungen an die Ukraine]. The speech should be in German.


In [62]:
# let's use the API and see if we might get better/different results
# example_prompt = prompt_speech_v2(pol = pol_party[0][0], par = pol_party[0][1], top = "Waffenlierferungen an die Ukraine", lng= "de")
example_prompt = prompt_speech_v2(pol = pol_party[0][0], par = pol_party[0][1], top = "Waffenlierferungen an die Ukraine", lng= "en")

print(example_prompt)
completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": example_prompt}],n=3)

Write a detailed speech of at least 1000 words for the German Bundestag in the style of the member of the Bundestag Alice Weidel from the AfD party on the topic of Waffenlierferungen an die Ukraine. Return only the complete speech created.


In [63]:
print(completion.keys())

print(completion['usage'])

completion['choices'][0]['message']['content']

responses = [choice['message']['content'] for choice in completion['choices']]

token_count = 0
for resp in responses:
    calculated_tokens = word_tokenize(resp, language='english')
    # print(len(calculated_tokens))
    print(resp)
    token_count += token_count + len(calculated_tokens) # this seems to overstate the actual count of needed completion_tokens (2050 calculated vs. 1385 completion tokens (3 runs))

print(token_count)

dict_keys(['id', 'object', 'created', 'model', 'choices', 'usage', 'system_fingerprint'])
{
  "prompt_tokens": 63,
  "completion_tokens": 3016,
  "total_tokens": 3079
}
Sehr geehrter Herr Präsident, liebe Kolleginnen und Kollegen,

ich stehe heute hier vor Ihnen, um über ein äußerst kontroverses Thema zu sprechen - die Waffenlieferungen an die Ukraine. Dieses Thema ist von großer Bedeutung, da es nicht nur die Sicherheit in Europa betrifft, sondern auch die Frage aufwirft, wie wir als deutsche Regierungspartei mit internationalen Konflikten umgehen sollten.

Der Vorschlag, Waffen an die Ukraine zu liefern, ist ein heikles Thema, das viele Menschen aus unterschiedlichen politischen Lagern polarisiert. Auf der einen Seite haben wir die Befürworter, die argumentieren, dass die Ukraine ein souveräner Staat ist, der das Recht hat, sich vor Aggressionen von außen zu verteidigen. Auf der anderen Seite haben wir jedoch die Gegner, die besorgt sind, dass die Lieferung von Waffen den Konflikt in